In [0]:
import nltk
import pandas as pd
import re

In [0]:
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown_tei.zip.
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_cat.zip.
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_esp.zip.
[nltk_data]    | Downloading package chat80 to /root/nltk_data...
[nltk_data]    |   Unzipp

True

In [0]:
# read json into a dataframe
df_idf=pd.read_json("/stackoverflow-test.json",lines=True)
df_idf.head()

,id,title,body,accepted_answer_id,answer_count,comment_count,creation_date,last_activity_date,last_edit_date,last_editor_display_name,last_editor_user_id,owner_display_name,owner_user_id,post_type_id,score,tags,view_count,favorite_count
0,3247246,Integrate War-Plugin for m2eclipse into Eclips...,<p>I set up a small web project with JSF and M...,3247526.0,2,0,2010-07-14 14:39:48.053 UTC,2010-07-14 16:02:19.683 UTC,2010-07-14 15:56:37.803 UTC,,70604.0,,389430.0,1,2,eclipse|maven-2|tomcat|m2eclipse,1653,NaN
1,40270764,phantomjs-node page.evaulate seems to hang,<p>I have an implementation of 'waitfor' with ...,NaN,1,0,2016-10-26 19:35:00.537 UTC,2016-11-02 20:05:09.143 UTC,NaN,,NaN,,245076.0,1,0,node.js|phantomjs,35,NaN
2,27532383,Dynamic operations can only be performed in ho...,<p>I'm working with an API that requires:</p>\...,NaN,1,0,2014-12-17 18:31:18.6 UTC,2014-12-17 19:57:43.443 UTC,NaN,,NaN,,3105880.0,1,1,c#|asp.net-mvc,4372,NaN
3,33511888,CSS with relative URL to background image?,<p>I have a file structure of:</p>\n\n<pre><co...,NaN,2,2,2015-11-04 00:50:35.223 UTC,2015-11-04 01:51:03.037 UTC,2015-11-04 01:51:03.037 UTC,,5464492.0,,5464492.0,1,0,css|background-image,406,NaN
4,46160163,Share canvas image on android,<p>Hello so I write a small game where in the ...,46160246.0,1,0,2017-09-11 16:19:18.32 UTC,2017-09-11 16:24:12.69 UTC,NaN,,NaN,,8570512.0,1,0,android|canvas|bitmap|share,52,NaN


In [0]:
# print schema
print("Schema:\n\n",df_idf.dtypes)
print("Number of questions,columns=",df_idf.shape)

Schema:

 id                            int64
title                        object
body                         object
accepted_answer_id          float64
answer_count                  int64
comment_count                 int64
creation_date                object
last_activity_date           object
last_edit_date               object
last_editor_display_name     object
last_editor_user_id         float64
owner_display_name           object
owner_user_id               float64
post_type_id                  int64
score                         int64
tags                         object
view_count                    int64
favorite_count              float64
dtype: object
Number of questions,columns= (500, 18)


In [0]:
df_idf.head(2)

,id,title,body,accepted_answer_id,answer_count,comment_count,creation_date,last_activity_date,last_edit_date,last_editor_display_name,last_editor_user_id,owner_display_name,owner_user_id,post_type_id,score,tags,view_count,favorite_count
0,3247246,Integrate War-Plugin for m2eclipse into Eclips...,<p>I set up a small web project with JSF and M...,3247526.0,2,0,2010-07-14 14:39:48.053 UTC,2010-07-14 16:02:19.683 UTC,2010-07-14 15:56:37.803 UTC,,70604.0,,389430.0,1,2,eclipse|maven-2|tomcat|m2eclipse,1653,NaN
1,40270764,phantomjs-node page.evaulate seems to hang,<p>I have an implementation of 'waitfor' with ...,NaN,1,0,2016-10-26 19:35:00.537 UTC,2016-11-02 20:05:09.143 UTC,NaN,,NaN,,245076.0,1,0,node.js|phantomjs,35,NaN


In [0]:
# Text pre-processing

import re
def pre_process(text):
    
    # lowercase
    text=text.lower()
    
    #remove tags
    text=re.sub("<!--?.*?-->","",text)
    
    # remove special characters and digits
    text=re.sub("(\\d|\\W)+"," ",text)
    
    return text

In [0]:
# combining 'title' and 'body' fields together
df_idf['text'] = df_idf['title'] + df_idf['body']

# pre-processing the content of the newly formed field
df_idf['text'] = df_idf['text'].apply(lambda x:pre_process(x))

df_idf['text']

0      integrate war plugin for m eclipse into eclips...
1      phantomjs node page evaulate seems to hang p i...
2      dynamic operations can only be performed in ho...
3      css with relative url to background image p i ...
4      share canvas image on android p hello so i wri...
                             ...                        
495    is there any net string format compatible func...
496    how to handle failed xpath lookup in msxml fro...
497    logo animation don t know where to begin p i m...
498    how to introduce a new variable in coq p i was...
499    facebook php sdk login and localhost p i am tr...
Name: text, Length: 500, dtype: object

In [0]:
df_idf['text'][0]

'integrate war plugin for m eclipse into eclipse project p i set up a small web project with jsf and maven now i want to deploy on a tomcat server is there a possibility to automate that like a button in eclipse that automatically deploys the project to tomcat p p i read about a the a href http maven apache org plugins maven war plugin rel nofollow noreferrer maven war plugin a but i couldn t find a tutorial how to integrate that into my process eclipse m eclipse p p can you link me to help or try to explain it thanks p '

In [0]:
df_idf['text'][1]

'phantomjs node page evaulate seems to hang p i have an implementation of waitfor with phantomjs node and it seems that the code sitepage evaluate code has a big lag compared to when it should evaluate true you ll see below that i m logging out the content value and the content logs with what should evaluate as true but this doesn t seem to occur for a good seconds or so after the fact p p any idea what would cause this delay or if there s a better way to evaluate p pre code let promise require bluebird let phantom require phantom let sitepage let phinstance phantom create then instance gt phinstance instance return instance createpage then page gt sitepage page return page open https thepiratebay org search game then status gt return waituntil function this returns the correct content after a short period while the evaluate ends up taking maybe s longer after this content should evaluate true sitepage property content then content gt console log content return sitepage evaluate functi

In [0]:
# stop words removal

from sklearn.feature_extraction.text import CountVectorizer
import re

# function to generate a distinct set of english stop words from a text file containing custom english stop words
# this list contains more stop words than sklearn stop words
def get_stop_words(stop_file_path):
    """load stop words """
    
    with open(stop_file_path, 'r', encoding="utf-8") as f:
        stopwords = f.readlines()
        stop_set = set(m.strip() for m in stopwords)
        return frozenset(stop_set)

#load a set of stop words
stopwords=get_stop_words("/stopwords")

#get the text column 
docs=df_idf['text'].tolist()

In [0]:
#create a vocabulary of words, 
#ignore words that appear in 85% of documents, 
#eliminate stop words
#limit size of vocabulary to 8500 words

cv=CountVectorizer(max_df=0.85,stop_words=stopwords,max_features=8500)
word_count_vector=cv.fit_transform(docs)

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['come', 'vis', 'viser', 'visest'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [0]:
# first 10 words from the vocabulary:

list(cv.vocabulary_.keys())[:10]

['integrate',
 'war',
 'plugin',
 'eclipse',
 'project',
 'set',
 'small',
 'web',
 'jsf',
 'maven']

In [0]:
# computing idf values

from sklearn.feature_extraction.text import TfidfTransformer
 
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)

TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)

In [0]:
# consolidating the feature names
feature_names=cv.get_feature_names()
len(feature_names)

# get the document that we want to extract keywords from
doc=docs[0]
doc

'integrate war plugin for m eclipse into eclipse project p i set up a small web project with jsf and maven now i want to deploy on a tomcat server is there a possibility to automate that like a button in eclipse that automatically deploys the project to tomcat p p i read about a the a href http maven apache org plugins maven war plugin rel nofollow noreferrer maven war plugin a but i couldn t find a tutorial how to integrate that into my process eclipse m eclipse p p can you link me to help or try to explain it thanks p '

In [0]:
#generate tf-idf for the given document
tf_idf_vector=tfidf_transformer.transform(cv.transform([doc]))

In [0]:
#the following function sorts the values in the vector while preserving the column index
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

In [0]:
print(tf_idf_vector.tocoo().col)
print(tf_idf_vector.tocoo().data)

[8204 8186 7782 7763 7653 7528 6891 6636 6622 6127 5987 5808 5781 5707
 5646 5645 5313 5069 5045 5031 4550 4251 4237 4001 3784 3481 3473 3380
 2710 2581 2307 1961 1958 1598  981  641  638  417]
[0.06931353 0.35282562 0.1042839  0.05848174 0.2508059  0.05449021
 0.09095927 0.05159569 0.06277131 0.05401553 0.06984023 0.20952068
 0.07763464 0.11760854 0.1042839  0.29626105 0.08113947 0.0542514
 0.06734017 0.05651559 0.43115497 0.07334506 0.03858364 0.11760854
 0.2508059  0.05522495 0.05078616 0.05159569 0.06277131 0.09648949
 0.47232054 0.12540295 0.11207832 0.10132061 0.06431    0.08542905
 0.11760854 0.10132061]


array([0.06931353, 0.35282562, 0.1042839 , 0.05848174, 0.2508059 ,
       0.05449021, 0.09095927, 0.05159569, 0.06277131, 0.05401553,
       0.06984023, 0.20952068, 0.07763464, 0.11760854, 0.1042839 ,
       0.29626105, 0.08113947, 0.0542514 , 0.06734017, 0.05651559,
       0.43115497, 0.07334506, 0.03858364, 0.11760854, 0.2508059 ,
       0.05522495, 0.05078616, 0.05159569, 0.06277131, 0.09648949,
       0.47232054, 0.12540295, 0.11207832, 0.10132061, 0.06431   ,
       0.08542905, 0.11760854, 0.10132061])

In [0]:
# extracting name of feature with the help of feature id (or col id)
feature_names[8204]

'web'

In [0]:
# get the feature names and tf-idf score of top n items

def extract_topn_from_vector(feature_names, sorted_items, topn=10):
        
    #use only topn items from vector
    sorted_items = sorted_items[:topn]
 
    score_vals = []
    feature_vals = []
    
    # word index and corresponding tf-idf score
    for idx, score in sorted_items:
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])
 
    #create a tuples of feature,score
    #results = zip(feature_vals,score_vals)
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results

In [0]:
#sort the tf-idf vectors by descending order of scores
sorted_items=sort_coo(tf_idf_vector.tocoo())
print(sorted_items)

tf_idf_vector.tocoo()

[(2307, 0.4723205415287291), (4550, 0.43115496835617034), (8186, 0.35282561622659103), (5645, 0.29626104682402304), (7653, 0.2508058996825358), (3784, 0.2508058996825358), (5808, 0.20952067887970735), (1961, 0.1254029498412679), (5707, 0.11760853874219701), (4001, 0.11760853874219701), (638, 0.11760853874219701), (1958, 0.11207831605797111), (7782, 0.10428390495890023), (5646, 0.10428390495890023), (1598, 0.10132060738841142), (417, 0.10132060738841142), (2581, 0.09648949385982934), (6891, 0.09095927117560346), (641, 0.08542904849137756), (5313, 0.08113947452244903), (5781, 0.07763463739230668), (4251, 0.07334506342337815), (5987, 0.06984022629323579), (8204, 0.06931352531639606), (5045, 0.06734016518806117), (981, 0.06431000360900989), (6622, 0.06277131197657511), (2710, 0.06277131197657511), (7763, 0.05848173800764659), (5031, 0.056515592509939004), (3481, 0.05522494797171183), (7528, 0.054490206955855484), (5069, 0.054251404095094026), (6127, 0.054015531404764394), (6636, 0.05159568

<1x8500 sparse matrix of type '<class 'numpy.float64'>'
	with 38 stored elements in COOrdinate format>

In [0]:
#extract only the top n; n here is 10
keywords=extract_topn_from_vector(feature_names,sorted_items,10)
keywords

{'deploys': 0.125,
 'eclipse': 0.472,
 'integrate': 0.251,
 'jsf': 0.118,
 'maven': 0.431,
 'plugin': 0.296,
 'possibility': 0.118,
 'project': 0.21,
 'tomcat': 0.251,
 'war': 0.353}

In [0]:
print("\n=====Doc=====")
print(doc)
print("\n===Keywords===")
for k in keywords:
    print(k,keywords[k])


=====Doc=====
integrate war plugin for m eclipse into eclipse project p i set up a small web project with jsf and maven now i want to deploy on a tomcat server is there a possibility to automate that like a button in eclipse that automatically deploys the project to tomcat p p i read about a the a href http maven apache org plugins maven war plugin rel nofollow noreferrer maven war plugin a but i couldn t find a tutorial how to integrate that into my process eclipse m eclipse p p can you link me to help or try to explain it thanks p 

===Keywords===
eclipse 0.472
maven 0.431
war 0.353
plugin 0.296
tomcat 0.251
integrate 0.251
project 0.21
deploys 0.125
possibility 0.118
jsf 0.118
